In [2]:
# Minimal imports needed for fetching and processing (no local audio dir, handle .m4a)
import os
from pathlib import Path
import tempfile
from urllib.parse import unquote

import numpy as np
import pandas as pd
import requests
import librosa
from dotenv import load_dotenv
from supabase import create_client

False


In [3]:

print("Current working directory:", os.getcwd())
print("Files Here:", os.listdir())
notebook_folder = os.path.dirname(os.path.abspath("__file__"))  # safer for scripts
dotenv_path = os.path.join(notebook_folder, ".env")

print("Looking for .env at:", dotenv_path)

load_dotenv(dotenv_path)

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

print("URL:", SUPABASE_URL)
print("KEY length:", len(SUPABASE_KEY) if SUPABASE_KEY else "None")


Current working directory: c:\Users\savir\MaterialClassifier\src
Files Here: ['.env', 'audio', 'audio_processing.py', 'models', 'train_model.ipynb', '_init_.py']
Looking for .env at: c:\Users\savir\MaterialClassifier\src\.env
URL: https://lapffcqqdubcyswirkye.supabase.co
KEY length: 208


In [4]:
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
table_name = "recordings_metadata"
response  = supabase.table(table_name).select("*").execute()
if response.data:
    df = pd.DataFrame(response.data)
    print("Data fetched successfully")
    print(df.head())
else:
    print("No data found in the table.")

Data fetched successfully
                                     id  \
0  6fcad7a6-a3eb-4ef1-bb30-36af8effa53a   

                                           file_name  \
0  Paper_Large_10in_Flat_Cardboard_2025-10-31T02-...   

                                            file_url description material  \
0  https://lapffcqqdubcyswirkye.supabase.co/stora...   Cardboard    Paper   

            size shape                      timestamp  \
0  Large (>10in)  Flat  2025-10-31T02:07:43.551+00:00   

                        created_at  \
0  2025-10-31T02:07:43.78475+00:00   

                                   ambient_file_name  ambient_duration_ms  
0  Paper_Large_10in_Flat_Cardboard_2025-10-31T02-...                 1500  


In [5]:
print(f"Rows: {len(df)}; Columns: {list(df.columns)}")


Rows: 1; Columns: ['id', 'file_name', 'file_url', 'description', 'material', 'size', 'shape', 'timestamp', 'created_at', 'ambient_file_name', 'ambient_duration_ms']


In [6]:
AMBIENT_DURATION_SEC = 1.5

def split_ambient_chirp(audio_array, sample_rate):
    data = audio_array
    sr = sample_rate
    if data.ndim > 1:
        data = data[:, 0]
    split_sample = int(sr * AMBIENT_DURATION_SEC)
    ambient = data[:split_sample]
    chirp = data[split_sample:]
    return ambient, chirp, sr

In [7]:
def spectral_subtract(ambient, chirp):
    fft_ambient = np.fft.fft(ambient)
    fft_chirp = np.fft.fft(chirp)
    subtracted = np.abs(fft_chirp) - np.abs(fft_ambient)
    subtracted[subtracted < 0] = 0
    return subtracted

In [ ]:
# Process m4a files directly from Supabase without storing in audio/
BUCKET_NAME = "recordings"  # change if your bucket is named differently

# --- Load recordings metadata from Supabase ---
response = supabase.table("recordings_metadata").select("*").execute()
df = pd.DataFrame(response.data)
print(f"✅ Loaded {len(df)} rows from recordings_metadata")

# --- Helper to load m4a to numpy using a temporary file and librosa ---
def load_m4a_from_bytes(m4a_bytes: bytes, sr: int | None = None):
    tmp = tempfile.NamedTemporaryFile(suffix=".m4a", delete=False)
    try:
        tmp.write(m4a_bytes)
        tmp.flush()
        tmp_path = tmp.name
    finally:
        tmp.close()  # important on Windows to release the handle before reading
    try:
        audio, sample_rate = librosa.load(tmp_path, sr=sr, mono=True)
    finally:
        try:
            os.remove(tmp_path)
        except Exception:
            pass
    return audio, sample_rate

# --- Stream one example through spectral subtraction (no saving) ---
processed = 0
for idx, row in df.iterrows():
    object_path = row.get("file_name")
    if not object_path:
        continue
    object_path = unquote(object_path)

    try:
        m4a_bytes = supabase.storage.from_(BUCKET_NAME).download(object_path)
    except Exception as e:
        # Fallback to signed URL
        try:
            signed = supabase.storage.from_(BUCKET_NAME).create_signed_url(object_path, 60)
            signed_url = signed.get("signedURL") or signed.get("signed_url") or signed.get("url")
            r = requests.get(signed_url, timeout=30)
            r.raise_for_status()
            m4a_bytes = r.content
        except Exception as e2:
            print(f"⚠️ Could not fetch {object_path}: {e2}")
            continue

    try:
        audio, sr = load_m4a_from_bytes(m4a_bytes, sr=None)
        ambient, chirp, _ = split_ambient_chirp(audio, sample_rate=sr)
        spec = spectral_subtract(ambient, chirp)
        print(f"✅ Spectral subtraction computed: sr={sr}, ambient={len(ambient)}, chirp={len(chirp)}, spec={len(spec)}")
        processed += 1
        break  # stop after first successful example as requested
    except Exception as e:
        print(f"❌ Failed processing {object_path}: {e}")

if processed == 0:
    print("No recordings processed. Check storage keys (file_name) and that ffmpeg is available for m4a decoding.")

✅ Loaded 1 rows from recordings_metadata
❌ Failed processing Paper_Large_10in_Flat_Cardboard_2025-10-31T02-07-42_full.m4a: [Errno 13] Permission denied: 'C:\\Users\\savir\\AppData\\Local\\Temp\\tmp6of4xtxq.m4a'
No recordings processed. Check storage keys (file_name) and that ffmpeg is available for m4a decoding.


C:\Users\savir\AppData\Local\Temp\ipykernel_3040\3171386576.py:14: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(tmp.name, sr=sr, mono=True)
c:\Users\savir\MaterialClassifier\venv\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [21]:
# Verify downloaded WAVs and run split + spectral subtraction on one example
from scipy.io.wavfile import read as wavread

wav_files = sorted([f for f in os.listdir(LOCAL_AUDIO_DIR) if f.lower().endswith('.wav')])
print("Found WAV files:", wav_files)

if wav_files:
    test_path = os.path.join(LOCAL_AUDIO_DIR, wav_files[0])
    print("Testing:", test_path)
    sr, data = wavread(test_path)
    ambient, chirp, sr2 = split_ambient_chirp(test_path)
    spec = spectral_subtract(ambient, chirp)
    print(f"Sample rate: {sr}, ambient len: {len(ambient)}, chirp len: {len(chirp)}, spec len: {len(spec)}")
else:
    print("No WAVs downloaded. Check logs above.")



Found WAV files: []
No WAVs downloaded. Check logs above.


In [22]:
# Build dataset: spectral-subtraction features per recording
from scipy.io.wavfile import read as wavread
from urllib.parse import unquote

features = []
labels = []
paths = []

K_BINS = 512  # number of frequency bins to keep

for idx, row in df.iterrows():
    object_path = row.get("file_name")
    if not object_path:
        continue
    object_path = unquote(object_path)
    local_path = os.path.join(LOCAL_AUDIO_DIR, os.path.basename(object_path))
    if not os.path.exists(local_path):
        # try download once more if missing
        try:
            wav_bytes = supabase.storage.from_(BUCKET_NAME).download(object_path)
            with open(local_path, "wb") as f:
                f.write(wav_bytes)
        except Exception:
            continue

    try:
        sr, data = wavread(local_path)
        if data.ndim > 1:
            data = data[:,0]
        ambient, chirp, _ = split_ambient_chirp(data, sample_rate=sr)
        # Use rFFT and subtract magnitudes
        spec_ambient = np.abs(np.fft.rfft(ambient))
        spec_chirp = np.abs(np.fft.rfft(chirp))
        spec_diff = np.clip(spec_chirp - spec_ambient, 0, None)
        spec_log = np.log1p(spec_diff)
        # Fix feature length to K_BINS
        if len(spec_log) >= K_BINS:
            feat = spec_log[:K_BINS]
        else:
            pad = np.zeros(K_BINS - len(spec_log))
            feat = np.concatenate([spec_log, pad])
        features.append(feat.astype(np.float32))
        labels.append(row.get("material"))
        paths.append(local_path)
    except Exception:
        continue

print(f"Built {len(features)} samples with {K_BINS} features each")


Built 0 samples with 512 features each


In [23]:
# Train/test split and simple classifier
if len(features) >= 2 and len(set(labels)) > 1:
    X = np.vstack(features)
    y = np.array(labels)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

    clf = Pipeline([
        ("scaler", StandardScaler()),
        ("logreg", LogisticRegression(max_iter=200))
    ])

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(classification_report(y_test, y_pred))
else:
    print("Not enough labeled samples to train a classifier.")


Not enough labeled samples to train a classifier.


In [24]:
# Save model if trained
import pickle
from pathlib import Path

models_dir = Path("models")
models_dir.mkdir(parents=True, exist_ok=True)

if 'clf' in globals() and len(features) >= 2 and len(set(labels)) > 1:
    model_path = models_dir / "material_classifier.pkl"
    with open(model_path, "wb") as f:
        pickle.dump(clf, f)
    print(f"Saved model to {model_path}")
else:
    print("Model not trained; skipping save.")


Model not trained; skipping save.
